In [2]:
import pandas as pd
import datetime as dt

In [3]:
activity = pd.read_csv('../datas/activity.csv')
canact_all = pd.read_csv('../datas/canact_all_preprocessed.csv')

In [4]:
act_df = activity.sort_values(['user_id','started_at'])
act_df.head(50)

act_df.index = pd.DatetimeIndex(act_df.started_at, name='started_at').tz_convert('Asia/Tokyo')
act_df['started_jp'] = act_df.index
act_df['started_date_jp'] = pd.to_datetime(act_df['started_jp']).dt.date
act_df=act_df[(act_df['started_date_jp']>=dt.date(2021,1,12))]
# act_df['hour'] = pd.to_datetime(act_df['datetime_jp']).dt.hour
act_df.index = pd.DatetimeIndex(act_df.finished_at, name='finished_at').tz_convert('Asia/Tokyo')
act_df['finished_jp'] = act_df.index
act_df['finished_date_jp'] = pd.to_datetime(act_df['finished_jp']).dt.date
act_df=act_df[(act_df['started_date_jp']>=dt.date(2021,1,12))]

act_df = act_df.reset_index(drop=True)

In [5]:
ques_morning_dams = pd.read_csv('../datas/preprocessed_questionaire/ques_morning_dams.csv')
ques_morning_dams['datetime_jp_before3'] = pd.to_datetime(ques_morning_dams['datetime_jp_before3'])
ques_morning_dams['datetime_jp_before6'] = pd.to_datetime(ques_morning_dams['datetime_jp_before6'])
ques_morning_dams['datetime_jp_before9'] = pd.to_datetime(ques_morning_dams['datetime_jp_before9'])
ques_morning_dams['datetime_jp_before12'] = pd.to_datetime(ques_morning_dams['datetime_jp'])-dt.timedelta(hours=12)
ques_morning_dams['datetime_jp_before24'] = pd.to_datetime(ques_morning_dams['datetime_jp'])-dt.timedelta(hours=24)
ques_morning_dams['datetime_jp'] = pd.to_datetime(ques_morning_dams['datetime_jp'])

In [6]:
act_df = act_df[['user_id','activity_type_id', 'activity', 'activity_group', 'Date',
       'Time', 'am_pm','started_jp','finished_jp', 'time_spent', 
       'record_type_id', 'name', 'value', 'meal', 'working_alone',
       'hobby_break', 'housework_childcare', 'break', 'transfer',
       'web_conference', 'other', 'work_outsideduty', 'facetoface_meeting',
       'collaboration', 'remote_conference', 'started_date_jp',
       'finished_date_jp']]

ques_morning_dams = ques_morning_dams[['user_id', 'value_dams', 
       'datetime_jp', 'Date_jp', 'hour', 'positive_score', 'depressive_score',
       'anxious_score', 'datetime_jp_before3', 'datetime_jp_before6',
       'datetime_jp_before9','datetime_jp_before12','datetime_jp_before24']]

act_df2 = act_df[['user_id','activity_type_id', 'activity', 'activity_group',
       'Time', 'am_pm','started_jp','finished_jp', 'time_spent', 
       'record_type_id', 'name', 'value']]

In [7]:
# 行動データ，アンケートデータのマージ
ques_act = pd.merge(ques_morning_dams, act_df2,on='user_id')
# ques_eval.loc[ques_eval['name']=='業務に集中できていた', 'syutyu'] =ques_eval['value_eval']

# タイムゾーンを消す
ques_act['datetime_jp_before3'] = ques_act['datetime_jp_before3'].dt.tz_localize(None)
ques_act['finished_jp'] = ques_act['finished_jp'].dt.tz_localize(None)
ques_act['datetime_jp'] = ques_act['datetime_jp'].dt.tz_localize(None)
ques_act['started_jp'] = ques_act['started_jp'].dt.tz_localize(None)
ques_act['datetime_jp_before6'] = ques_act['datetime_jp_before6'].dt.tz_localize(None)
ques_act['datetime_jp_before9'] = ques_act['datetime_jp_before9'].dt.tz_localize(None)
ques_act['datetime_jp_before12'] = ques_act['datetime_jp_before12'].dt.tz_localize(None)
ques_act['datetime_jp_before24'] = ques_act['datetime_jp_before24'].dt.tz_localize(None)

In [8]:
#過去３時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before3']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before3'] = ques_act['finished_jp']-ques_act['datetime_jp_before3']
ques_act.loc[(ques_act['datetime_jp_before3']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before3'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before3']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp'])&(ques_act['datetime_jp_before3']<=ques_act['started_jp']),'act_before3'] = ques_act['datetime_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before3'])&(ques_act['datetime_jp']<=ques_act['finished_jp']),'act_before3'] = ques_act['datetime_jp']-ques_act['datetime_jp_before3']

#過去３〜６時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before6']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before3']),'act_before6'] = ques_act['finished_jp']-ques_act['datetime_jp_before6']
ques_act.loc[(ques_act['datetime_jp_before6']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before3']),'act_before6'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before6']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp_before3'])&(ques_act['datetime_jp_before6']<=ques_act['started_jp']),'act_before6'] = ques_act['datetime_jp_before3']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before6'])&(ques_act['datetime_jp_before3']<=ques_act['finished_jp']),'act_before6'] = ques_act['datetime_jp_before3']-ques_act['datetime_jp_before6']

#過去6〜9時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before9']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before6']),'act_before9'] = ques_act['finished_jp']-ques_act['datetime_jp_before9']
ques_act.loc[(ques_act['datetime_jp_before9']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before6']),'act_before9'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before9']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp_before6'])&(ques_act['datetime_jp_before9']<=ques_act['started_jp']),'act_before9'] = ques_act['datetime_jp_before6']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before9'])&(ques_act['datetime_jp_before9']<=ques_act['finished_jp']),'act_before9'] = ques_act['datetime_jp_before6']-ques_act['datetime_jp_before9']

#過去6〜9時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before12']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before9']),'act_before12'] = ques_act['finished_jp']-ques_act['datetime_jp_before12']
ques_act.loc[(ques_act['datetime_jp_before12']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before9']),'act_before12'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before12']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp_before9'])&(ques_act['datetime_jp_before12']<=ques_act['started_jp']),'act_before12'] = ques_act['datetime_jp_before9']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before12'])&(ques_act['datetime_jp_before12']<=ques_act['finished_jp']),'act_before12'] = ques_act['datetime_jp_before9']-ques_act['datetime_jp_before12']

#過去24時間分の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before24']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before24'] = ques_act['finished_jp']-ques_act['datetime_jp_before24']
ques_act.loc[(ques_act['datetime_jp_before24']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before24'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before24']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp'])&(ques_act['datetime_jp_before24']<=ques_act['started_jp']),'act_before24'] = ques_act['datetime_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before24'])&(ques_act['datetime_jp']<=ques_act['finished_jp']),'act_before24'] = ques_act['datetime_jp']-ques_act['datetime_jp_before24']

In [9]:
ques_act_user7 = ques_act.query('user_id == 7')


# 前日の行動を集計

In [10]:
ques_act_shaping = ques_act[['user_id', 'activity_group','activity_type_id','activity','value_dams', 'datetime_jp', 
       'positive_score', 'depressive_score', 'anxious_score', 'started_jp',
       'finished_jp', 'time_spent', 'record_type_id', 'name',
       'act_before3', 'act_before6', 'act_before9', 'act_before12',
       'act_before24']]

In [11]:
ques_act_shaping1 = ques_act[['user_id', 'started_jp','Date_jp',
       'finished_jp',
       'act_before3', 'act_before6', 'act_before9', 'act_before12',
       'act_before24']]
ques_act_shaping['activity'].unique()

array(['食事（業務時間外）', 'その他', '移動', '単独作業', '休憩（業務時間内）', '対面会議', '趣味・休憩',
       'Web会議', '共同作業（コミュニケーションを伴う）', '家事・育児', '業務外作業', '電話（会議）'],
      dtype=object)

In [12]:
ques_act_shaping['activity_type_id'].unique()

array([10., 12., 11.,  5.,  7.,  1.,  8.,  2.,  4.,  9.,  6.,  3.])

In [13]:
ques_act_shaping=ques_act_shaping.groupby(['user_id','started_jp','activity']).mean()
# ques_act_shaping['user_id'] = ques_act_shaping.index
finised_jp = ques_act[['user_id', 'started_jp','finished_jp',
       'act_before3', 'act_before6', 'act_before9', 'act_before12',
       'act_before24']]
ques_act_shaping2 = ques_act_shaping.reset_index()
ques_act_shaping = pd.merge(ques_act_shaping2,finised_jp,on=['user_id','started_jp'],left_on)

SyntaxError: positional argument follows keyword argument (<ipython-input-13-71220319697b>, line 7)

In [52]:
for user_id,dams_date in zip(ques_morning_dams['user_id'],ques_morning_dams['datetime_jp']):
    for user_id_act, finished_date, activity_type_id,time_spent in zip(act_df['user_id'],act_df['finished_jp'],act_df['activity_type_id'],act_df  
        

SyntaxError: unexpected EOF while parsing (<ipython-input-52-5cea3da31b52>, line 3)

In [71]:
#　アンケートから24時間以内の対面会議の時間を集計
ques_morning_dams['meeting_ftof'] = 0
for user_id_act, finished_date, activity_type_id,time_spent in zip(act_df['user_id'],act_df['finished_jp'],act_df['activity_type_id'],act_df['time_spent']):
    ques_morning_dams.loc[(activity_type_id == 1)&(user_id_act==ques_morning_dams['user_id'])&(finished_date<ques_morning_dams['datetime_jp'])&(ques_morning_dams['datetime_jp_before24']<finished_date), 'meeting_ftof'] = time_spent

In [137]:
pd.to_timedelta(ques_morning_dams.query('meeting_ftof != 0 and user_id == 77')['meeting_ftof']).apply(timedelta_to_minutes)

119      81.383333
395      15.616667
578      16.716667
648      20.716667
722      75.651617
851      29.416667
1085    121.650000
1165     84.000000
1311    109.883333
Name: meeting_ftof, dtype: float64

In [135]:
act_df['meal'] = pd.to_timedelta(act_df['meal']).apply(timedelta_to_minutes)
act_df['working_alone'] = pd.to_timedelta(act_df['working_alone']).apply(timedelta_to_minutes)
act_df['hobby_break'] = pd.to_timedelta(act_df['hobby_break']).apply(timedelta_to_minutes)
act_df['housework_childcare'] = pd.to_timedelta(act_df['housework_childcare']).apply(timedelta_to_minutes)
act_df['break'] = pd.to_timedelta(act_df['break']).apply(timedelta_to_minutes)
act_df['transfer'] = pd.to_timedelta(act_df['transfer']).apply(timedelta_to_minutes)
act_df['web_conference'] = pd.to_timedelta(act_df['web_conference']).apply(timedelta_to_minutes)
act_df['other'] = pd.to_timedelta(act_df['other']).apply(timedelta_to_minutes)
act_df['work_outsideduty'] = pd.to_timedelta(act_df['work_outsideduty']).apply(timedelta_to_minutes)
act_df['facetoface_meeting'] = pd.to_timedelta(act_df['facetoface_meeting']).apply(timedelta_to_minutes)
act_df['collaboration'] = pd.to_timedelta(act_df['collaboration']).apply(timedelta_to_minutes)
act_df['remote_conference'] = pd.to_timedelta(act_df['remote_conference']).apply(timedelta_to_minutes)

In [133]:
def timedelta_to_minutes(td):
    sec=td.total_seconds()
    return sec/60
act_df['remote_conference'].apply(timedelta_to_minutes)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
33437    0.0
33438    0.0
33439    0.0
33440    0.0
33441    0.0
Name: remote_conference, Length: 33442, dtype: float64

In [147]:
act_df.sort_values(['user_id','started_jp','activity_type_id'])

,user_id,activity_type_id,activity,activity_group,Date,Time,am_pm,started_jp,finished_jp,time_spent,...,break,transfer,web_conference,other,work_outsideduty,facetoface_meeting,collaboration,remote_conference,started_date_jp,finished_date_jp
0,1,4.0,共同作業（コミュニケーションを伴う）,業務関係,2021-01-30,22,1,2021-01-31 07:20:26.166000+09:00,2021-01-31 07:20:26.366000+09:00,0 days 00:00:00.200000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.003333,0.0,2021-01-31,2021-01-31
1,1,4.0,共同作業（コミュニケーションを伴う）,業務関係,2021-01-30,22,1,2021-01-31 07:32:00.585000+09:00,2021-01-31 07:32:00.952000+09:00,0 days 00:00:00.367000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.006117,0.0,2021-01-31,2021-01-31
2,2,11.0,移動,業務外関係,2021-01-12,23,1,2021-01-13 07:41:47+09:00,2021-01-13 08:55:17+09:00,0 days 01:13:30,...,0.0,73.500000,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-13,2021-01-13
3,2,5.0,単独作業,業務関係,2021-01-13,0,0,2021-01-13 08:55:21+09:00,2021-01-13 09:26:42+09:00,0 days 00:31:21,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-13,2021-01-13
4,2,5.0,単独作業,業務関係,2021-01-13,0,0,2021-01-13 08:55:21+09:00,2021-01-13 09:26:42+09:00,0 days 00:31:21,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-13,2021-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33437,104,5.0,単独作業,業務関係,2021-01-29,8,0,2021-01-29 16:12:12+09:00,2021-01-29 17:13:38+09:00,0 days 01:01:26,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-29,2021-01-29
33438,104,5.0,単独作業,業務関係,2021-01-29,8,0,2021-01-29 16:12:12+09:00,2021-01-29 17:13:38+09:00,0 days 01:01:26,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-29,2021-01-29
33439,104,11.0,移動,業務外関係,2021-01-29,10,0,2021-01-29 17:14:03+09:00,2021-01-29 19:29:58+09:00,0 days 02:15:55,...,0.0,135.916667,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-29,2021-01-29
33440,104,10.0,食事（業務時間外）,業務外関係,2021-01-29,12,1,2021-01-29 19:59:28+09:00,2021-01-29 21:18:34+09:00,0 days 01:19:06,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2021-01-29,2021-01-29


In [177]:
act_df2 = act_df[['user_id', 'activity_type_id', 'activity', 'activity_group', 'Date',
       'started_jp', 'finished_jp', 'time_spent',
       'meal', 'working_alone',
       'hobby_break', 'housework_childcare', 'break', 'transfer',
       'web_conference', 'other', 'work_outsideduty', 'facetoface_meeting',
       'collaboration', 'remote_conference', 'started_date_jp',
       'finished_date_jp']]


In [186]:
act_df2['hour'] = act_df2['finished_jp'].dt.hour
act_df2['am_pm'] = 0
act_df2.loc[act_df2['hour']>=12,'am_pm'] = 1
act_df2 = act_df2[['user_id', 'activity_type_id', 'activity', 'activity_group', 'Date',
       'started_jp', 'finished_jp', 'time_spent', 'hour', 'am_pm', 'meal', 'working_alone',
       'hobby_break', 'housework_childcare', 'break', 'transfer',
       'web_conference', 'other', 'work_outsideduty', 'facetoface_meeting',
       'collaboration', 'remote_conference', 'started_date_jp',
       'finished_date_jp']]

<ipython-input-186-a86e40b1351e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act_df2['hour'] = act_df2['finished_jp'].dt.hour
<ipython-input-186-a86e40b1351e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act_df2['am_pm'] = 0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [197]:
act_df2 = act_df2.drop_duplicates()
act_time_day = act_df2.groupby(['user_id','Date']).sum()
act_time_ampm = act_df2.groupby(['user_id','Date','am_pm']).sum()
act_time_day_count = act_df2.groupby(['user_id','Date']).count()
act_time_ampm_count = act_df2.groupby(['user_id','Date','am_pm']).count()
act_time_ampm = act_time_ampm.reset_index()
act_time_day = act_time_day.reset_index()
act_time_day_count = act_time_day_count.reset_index()
act_time_ampm_count = act_time_ampm_count.reset_index()

In [207]:
from datetime import timedelta
act_time_ampm['Days_plus1'] = pd.to_datetime(act_time_ampm['Date']) + timedelta(days=1)
act_time_day['Days_plus1'] = pd.to_datetime(act_time_day['Date']) + timedelta(days=1)
act_time_day_count['Days_plus1'] = pd.to_datetime(act_time_day_count['Date']) + timedelta(days=1)
act_time_ampm_count['Days_plus1'] = pd.to_datetime(act_time_ampm_count['Date']) + timedelta(days=1)


In [222]:
ques_morning_dams['Date_jp'] = pd.to_datetime(ques_morning_dams['Date_jp'])
ques_act_day = pd.merge(ques_morning_dams, act_time_day, left_on=['user_id','Date_jp'],right_on=['user_id','Days_plus1'],how='inner')
ques_act_ampm = pd.merge(ques_morning_dams, act_time_ampm, left_on=['user_id','Date_jp'],right_on=['user_id','Days_plus1'],how='inner')

In [225]:
# ques_act_day.to_csv('../datas/ques_act_merge/ques_act_day.csv')
# ques_act_ampm.to_csv('../datas/ques_act_merge/ques_act_ampm.csv')